In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Checking information of the train data
train_df.info()

# Initialize LabelEncoder and dictionary to store encoders for each column
label_encoders = {}

# List of categorical columns to encode
categorical_columns = ['model', 'motor_type', 'wheel', 'color', 'status', 'type']

# Apply LabelEncoder to each categorical column for both train and test
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    train_df[col] = label_encoders[col].fit_transform(train_df[col])
    test_df[col] = test_df[col].apply(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

# Function to convert running values from 'km' to 'miles'
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])
        miles = kilometers * 0.621371
        return f"{miles:.2f} miles"
    elif 'miles' in value:
        return value
    else:
        return value 

# Apply the conversion for both train and test datasets
train_df['running'] = train_df['running'].apply(convert_running)
test_df['running'] = test_df['running'].apply(convert_running)

# Clean and convert 'running' column to numeric values
train_df['running'] = train_df['running'].str.replace('miles', '').str.strip()
test_df['running'] = test_df['running'].str.replace('miles', '').str.strip()
train_df['running'] = pd.to_numeric(train_df['running'], errors='coerce').fillna(0).astype('int64')
test_df['running'] = pd.to_numeric(test_df['running'], errors='coerce').fillna(0).astype('int64')

# Drop duplicates from the training data
train_df = train_df.drop_duplicates()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [12]:

# Prepare features and target variable
X = train_df.drop(['price', 'wheel'], axis = 1) # Replace 'target_column' with your actual target column name
y = train_df['price']  # Your target variable

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




In [14]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df.drop(columns=['Id', 'wheel']))  # Adjust as necessary


In [16]:

# Define the deep learning model with improved architecture
model = keras.Sequential([
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1)  # Output layer for regression
])


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:

# Compile the model with a learning rate scheduler
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mae'])


In [20]:

# Fit the model with early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)


In [22]:

history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), 
                    epochs=200, batch_size=32, callbacks=[early_stopping], verbose=1)



Epoch 1/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 15975.6973 - mae: 15973.1016 - val_loss: 15867.6680 - val_mae: 15864.9561
Epoch 2/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16153.4922 - mae: 16150.3291 - val_loss: 15109.6113 - val_mae: 15104.0869
Epoch 3/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14095.8584 - mae: 14088.8828 - val_loss: 11207.1699 - val_mae: 11194.3516
Epoch 4/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9044.6006 - mae: 9029.2041 - val_loss: 4077.6743 - val_mae: 4054.7783
Epoch 5/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4338.5298 - mae: 4315.4819 - val_loss: 3622.4961 - val_mae: 3600.2366
Epoch 6/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3872.0859 - mae: 3849.7834 - val_loss: 3399.0857 - val_mae: 3376.7224
Epoch 7/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3426.2930 - mae: 3404.0198 - val_loss: 3254.9622 - val_mae: 3232.3992
Epoch 8/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3564.5566 - mae: 3542.0002

In [25]:
# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)
print(f'Validation Mean Absolute Error: {val_mae:.2f}')


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2939.8088 - mae: 2919.4185
Validation Mean Absolute Error: 2848.71


In [27]:

# Make predictions on the test set
y_predict = model.predict(X_test_scaled)



13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [29]:
# Create a DataFrame for the predictions
df_predictions = pd.DataFrame(y_predict, columns=['price'])

# Display predictions
print(df_predictions)



            price
0    18868.279297
1    17014.986328
2    18653.800781
3    15579.852539
4     7351.168457
..            ...
406  22412.291016
407  13049.485352
408   9771.448242
409  17525.728516
410  12831.936523

[411 rows x 1 columns]


In [32]:
combined_df = pd.concat([test_df['Id'], df_predictions], axis=1)

combined_df

,Id,price
0,0,18868.279297
1,1,17014.986328
2,2,18653.800781
3,3,15579.852539
4,4,7351.168457
...,...,...
406,406,22412.291016
407,407,13049.485352
408,408,9771.448242
409,409,17525.728516


In [35]:
combined_df.to_csv('piiiii.csv', index=False)